In [1]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 

In [29]:
df = pd.read_csv("insurance_data.csv")
df.head()


,age,affordability,have_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordability']],df.have_insurance,test_size=0.2, random_state=25)

In [31]:
X_train

,age,affordability
0,22,1
13,29,0
6,55,0
17,58,1
24,50,1
19,18,1
25,54,1
16,25,0
20,21,1
3,52,0


In [32]:
len(X_train)

21

In [34]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [35]:
X_train_scaled

,age,affordability
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [36]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=1500)

Epoch 1/1500
1/1 [==============================] - 0s 463ms/step - loss: 0.6529 - accuracy: 0.5714
Epoch 2/1500
1/1 [==============================] - 0s 9ms/step - loss: 0.6526 - accuracy: 0.5714
Epoch 3/1500
1/1 [==============================] - 0s 11ms/step - loss: 0.6524 - accuracy: 0.5714
Epoch 4/1500
1/1 [==============================] - 0s 11ms/step - loss: 0.6522 - accuracy: 0.5714
Epoch 5/1500
1/1 [==============================] - 0s 12ms/step - loss: 0.6519 - accuracy: 0.5714
Epoch 6/1500
1/1 [==============================] - 0s 11ms/step - loss: 0.6517 - accuracy: 0.5714
Epoch 7/1500
1/1 [==============================] - 0s 11ms/step - loss: 0.6514 - accuracy: 0.5714
Epoch 8/1500
1/1 [==============================] - 0s 11ms/step - loss: 0.6512 - accuracy: 0.5714
Epoch 9/1500
1/1 [==============================] - 0s 15ms/step - loss: 0.6510 - accuracy: 0.5714
Epoch 10/1500
1/1 [==============================] - 0s 13ms/step - loss: 0.6507 - accuracy: 0.5714
Epoch 11/

In [37]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 119ms/step - loss: 0.5330 - accuracy: 0.6667


[0.5329789519309998, 0.6666666865348816]

In [38]:
X_test_scaled

,age,affordability
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [39]:
model.predict(X_test_scaled)

array([[0.7168118 ],
       [0.55953896],
       [0.40314674],
       [0.6170436 ],
       [0.72636294],
       [0.77928776]], dtype=float32)

In [40]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: have_insurance, dtype: int64

In [42]:
coef, intercept = model.get_weights()
coef, intercept

(array([[2.3772776],
        [0.8218439]], dtype=float32), array([-1.0104629], dtype=float32))

In [44]:
def sigmoid(x):
  import math 
  return 1/(1+math.exp(-x))
sigmoid(15)  

0.999999694097773

In [46]:
sigmoid(18)

0.9999999847700205

In [65]:
def prediction_function(age, affordability):
  weighted_sum = coef[0]*age + coef[1]*affordability + intercept
  return sigmoid(weighted_sum)

In [66]:
prediction_function(.47,1)

0.7168117778072501

In [67]:
prediction_function(.18,1)

0.5595389376905157

In [93]:
def sigmoid_numpy(X):
  return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))  

array([0.99999386, 0.5       , 0.73105858])

In [94]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new =  np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1- y_true)*np.log(1- y_predicted_new))


In [100]:
def gradient_descent(age, affordability, y_true, epochs):
    w1 = w2 =1 
    bias = 0 
    rate = 0.5
    n = len(age)
    for i in range(epochs):
      weighted_sum =  w1*age+w2*affordability +bias 
      y_predicted = sigmoid_numpy(weighted_sum)
      
      loss = log_loss(y_true, y_predicted)

      w1d =  (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
      w2d = (1/n)*np.dot(np.transpose(affordability), (y_predicted - y_true))
      bias_d = np.mean(y_predicted - y_true)
     
      w1 = w1-rate*w1d
      w2 = w2- rate*w2d
      bias = bias - rate*bias_d
      print(f'Epoch:{i}, w1:{w1} , w2:{w2}, bias:{bias}, loss:{loss}')
      

      return w1, w2, bias   

In [101]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordability'], y_train,1000)

Epoch:0, w1:0.9890295453116531 , w2:0.965210718860606, bias:-0.07568783640990467, loss:0.6528884040967335


(0.9890295453116531, 0.965210718860606, -0.07568783640990467)